# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# ML
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

# Loading data

In [4]:
# Load sales and market data
sales_df = pd.read_csv('../Data/Case2_Sales data.csv', sep=";")
market_df = pd.read_excel('/Users/diogocarvalho/Documents/GitHub/Case-2--Siemens-Sales-Forecast/Data/Case2_Market data.xlsx', header=[0,1,2])

# Preprocessing sales data

In [8]:
# Then try converting with 'to_numeric'. 
# 'errors="coerce"' will replace anything unparseable with NaN
sales_df['Sales_EUR'] = (
    sales_df['Sales_EUR']
    .replace(',','.', regex=True)        # if commas are being used as decimals
    .replace(r'[^0-9\.\-]', '', regex=True)  # remove any characters that aren’t digits, dot, or minus
    .pipe(pd.to_numeric, errors='coerce')
)

In [9]:
sales_df['Sales_EUR'].dtype

dtype('float64')

DATE is object, and must be converted to datetime

In [10]:
# Convert the DATE column to datetime
sales_df['DATE'] = pd.to_datetime(sales_df['DATE'], format='%d.%m.%Y')

In [11]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9802 entries, 0 to 9801
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATE        9802 non-null   datetime64[ns]
 1   Mapped_GCK  9802 non-null   object        
 2   Sales_EUR   9795 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 229.9+ KB


In [ ]:
sales_df.head()

,DATE,Mapped_GCK,Sales_EUR
0,01.10.2018,#1,0
1,02.10.2018,#1,0
2,03.10.2018,#1,0
3,04.10.2018,#1,0
4,05.10.2018,#1,0


In [13]:
# Aggregate to monthly level
sales_df['YearMonth'] = sales_df['DATE'].dt.to_period('M')
monthly_sales = sales_df.groupby(['YearMonth', 'Mapped_GCK'])['Sales_EUR'].sum().reset_index()

# Convert YearMonth to datetime
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].dt.to_timestamp()

In [14]:
sales_df.sample(10)

,DATE,Mapped_GCK,Sales_EUR,YearMonth
1390,2022-03-17,#11,0.00,2022-03
2452,2019-06-10,#8,68958.37,2019-06
8149,2019-11-28,#6,0.00,2019-11
4132,2020-03-16,#3,0.00,2020-03
9005,2019-07-30,#6,0.00,2019-07
6594,2019-12-09,#8,0.00,2019-12
4362,2019-07-12,#4,72365.29,2019-07
6771,2020-06-04,#6,0.00,2020-06
233,2018-12-06,#1,0.00,2018-12
5435,2019-07-22,#4,0.00,2019-07


# Preprocessing market data

In [16]:
market_df

Unnamed: 0_level_0  \
    Index 2010=100 (if not otherwise noted)   
                                       date   
0                                    2004m2   
1                                    2004m3   
2                                    2004m4   
3                                    2004m5   
4                                    2004m6   
..                                      ...   
214                                 2021m12   
215                                  2022m1   
216                                  2022m2   
217                                  2022m3   
218                                  2022m4   

                                       China  \
    Production Index Machinery & Electricals   
                              MAB_ELE_PRO156   
0                                  16.940704   
1                                  23.711852   
2                                  24.435235   
3                                  23.708115   
4                                  27.009138   
..                                       ...   
214                               310.763183   
215                               235.956129   
216                               235.956129   
217                               329.413367   
218                               267.373145   

                                             \
    Shipments Index Machinery & Electricals   
                             MAB_ELE_SHP156   
0                                 16.940704   
1                                 23.711852   
2                                 24.435235   
3                                 23.708115   
4                                 27.009138   
..                                      ...   
214                              310.763183   
215                              235.956129   
216                              235.956129   
217                              329.413367   
218                              267.373145   

                                      France  \
    Production Index Machinery & Electricals   
                              MAB_ELE_PRO250   
0                                 112.091273   
1                                 136.327976   
2                                 117.791806   
3                                 109.002541   
4                                 133.785737   
..                                       ...   
214                               100.565744   
215                                85.743503   
216                                90.603540   
217                               107.843548   
218                                87.698110   

                                             \
    Shipments Index Machinery & Electricals   
                             MAB_ELE_SHP250   
0                                 83.458866   
1                                106.168192   
2                                 92.007646   
3                                 85.696486   
4                                106.641482   
..                                      ...   
214                              134.589504   
215                              108.156320   
216                              117.715770   
217                              136.858720   
218                              116.528738   

                                     Germany  \
    Production Index Machinery & Electricals   
                              MAB_ELE_PRO276   
0                                  82.623037   
1                                 100.556582   
2                                  89.653203   
3                                  86.880571   
4                                  99.010814   
..                                       ...   
214                               118.103281   
215                                94.550610   
216                               103.987916   
217                               121.308119   
218                                99.522205   

                             

# Merge market data

In [ ]:
# Ensure macro data is monthly
market_df['YearMonth'] = market_df['Date'].dt.to_period('M').dt.to_timestamp()

# Merge with sales
full_df = pd.merge(monthly_sales, market_df, on='YearMonth', how='left')

# Sort for time series modeling
full_df = full_df.sort_values(['Mapped_GCK', 'YearMonth']).reset_index(drop=True)

# Feature Engineering

In [ ]:
# Example: Add lag and rolling mean for each product group
for lag in [1, 2, 3]:
    full_df[f'lag_{lag}'] = full_df.groupby('Mapped_GCK')['Sales EUR'].shift(lag)

for window in [3, 6]:
    full_df[f'rolling_mean_{window}'] = full_df.groupby('Mapped_GCK')['Sales EUR'].transform(lambda x: x.shift(1).rolling(window).mean())

# Model with XGBoost (ML Approach)

In [ ]:
results = []

# Loop through each product
for gck in full_df['Mapped_GCK'].unique():
    gck_df = full_df[full_df['Mapped_GCK'] == gck].dropna()
    
    train = gck_df[gck_df['YearMonth'] < '2022-05-01']
    test = gck_df[gck_df['YearMonth'] >= '2022-05-01']
    
    features = [col for col in gck_df.columns if col not in ['Sales EUR', 'Date', 'YearMonth', 'Mapped_GCK']]
    
    model = XGBRegressor(n_estimators=100, learning_rate=0.1)
    model.fit(train[features], train['Sales EUR'])
    
    preds = model.predict(test[features])
    
    rmse = np.sqrt(mean_squared_error(test['Sales EUR'], preds))
    results.append((gck, rmse))
    
    # Save predictions for submission
    test['Predicted_Sales'] = preds
    test[['YearMonth', 'Mapped_GCK', 'Predicted_Sales']].to_csv(f'predictions_{gck}.csv', index=False)

# Model with SARIMAX (Statistical Approach)

In [15]:
# Example for 1 product
gck = 'Product_X'
gck_df = full_df[full_df['Mapped_GCK'] == gck].set_index('YearMonth')

endog = gck_df['Sales EUR']
exog = gck_df[market_df.columns.difference(['Date', 'YearMonth'])]

# Train/test split
train_endog = endog[:'2022-04']
test_endog = endog['2022-05':]

train_exog = exog.loc[:'2022-04']
test_exog = exog.loc['2022-05':]

model = SARIMAX(train_endog, exog=train_exog, order=(1,1,1), seasonal_order=(1,1,1,12))
sarimax_res = model.fit()

forecast = sarimax_res.predict(start=test_endog.index[0], end=test_endog.index[-1], exog=test_exog)
rmse = np.sqrt(mean_squared_error(test_endog, forecast))
print(f"RMSE for {gck}: {rmse}")

NameError: name 'full_df' is not defined